# Simultaneous global and target analysis of the timeresolved fluorescence of WL-PSI of CF9212

## Step 1a Defining datasets and inspect data

In [ ]:
from __future__ import annotations

from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras import (
    plot_overview,
    plot_data_overview,
    plot_fitted_traces,
    select_plot_wavelengths,
)

The code below defined the (groups of) datasets used in the analysis. Only for a single dataset the plot_data_overview is shown to avoid repetition, it is left as an execirse to the reader to inspect the other data.

In [ ]:
# order FRLtr1, WLtr1, FRLtr2, WLtr2, FRLtr4
CF9212DATA_PATH1 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2a.ascii"
CF9212DATA_PATH2 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2b.ascii"
CF9212DATA_PATH3 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2c.ascii"
CF9212DATA_PATH4 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2d.ascii"
CF9212DATA_PATH5 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2e.ascii"
CF9212DATA_PATH6 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2f.ascii"

# to plot uncomment the next lines with "Ctrl /"
plot_data_overview(
    CF9212DATA_PATH2,
    nr_of_data_svd_vectors=4,
    linlog=False,
    linthresh=10,
    irf_location=57,
    title="WL-PSI Time Range 1",
    use_svd_number=True,
)
# plot_data_overview(CF9212DATA_PATH2, nr_of_data_svd_vectors=4, linlog=True, linthresh=30,irf_location=57);
# etc

In [ ]:
plot_data_overview(
    CF9212DATA_PATH4,
    nr_of_data_svd_vectors=4,
    linlog=False,
    linthresh=10,
    irf_location=57,
    title="WL-PSI Time Range 2",
    use_svd_number=True,
)

In [ ]:
plot_data_overview(
    CF9212DATA_PATH6,
    nr_of_data_svd_vectors=4,
    linlog=False,
    linthresh=100,
    irf_location=214,
    title="WL-PSI Time Range 4",
    use_svd_number=True,
)

## Step 1b Motivation of the simple model

With all three time ranges the SVD points to rank 4 matrix, which will require at least four temporally and spectrally linearly independent components. Since there is preknowledge that some free Chla is present we will start the global analysis with a sequential scheme with three compartments, plus a parallelly decaying free Chla compartment. 
Especially with the shortest time range a parabolic wavelength dependence of the rise of the IRF is visible, therefore we will model this dispersion with a 2nd order polynomial.
We will start with a simple Gaussian shaped IRF.

### Optional: display model file and starting parameters

#### Model file

In [ ]:
# Uncomment the following 3 lines to display the target model file in the notebook
from glotaran.utils.ipython import display_file

target_model_path = "models/globalWLstep1b_PSI_CF9212_streak_global_dispWLsimple.yml"
display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

#### Parameters file

In [ ]:
# Uncomment the following 3 lines and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters
target_parameters_path = (
    "models/globalWLstep1b_PSI_CF9212_streak_global_dispWLsimple.csv"
)
optimizedparameters = load_parameters(target_parameters_path)
optimizedparameters

## Step 1b Create scheme with simple Gaussian IRF and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/globalWLstep1b_PSI_CF9212_streak_global_dispWLsimple.yml",
    parameters="models/globalWLstep1b_PSI_CF9212_streak_global_dispWLsimple.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result1 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 7         2.0822e+02


In [ ]:
target_result1

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual
from pyglotaran_extras.plotting.style import ColorCode as cc


def plot_svd_of_residual(
    result_dataset,
    result_dataset2,
    result_dataset3,
    linlog,
    linthresh,
    loc1,
    loc2,
    loc3,
    colors=["tab:grey", "tab:orange", cc.cyan],
):
    fig, axes = plt.subplots(1, 2, figsize=(10, 2))
    custom_cycler = cycler(color=[colors[0]])
    plot_lsv_residual(
        result_dataset,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
        irf_location=loc1,
    )
    custom_cycler = cycler(color=[colors[1]])
    plot_lsv_residual(
        result_dataset2,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
        irf_location=loc2,
    )
    custom_cycler = cycler(color=[colors[2]])
    plot_lsv_residual(
        result_dataset3,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
        irf_location=loc3,
    )
    axes[0].set_xlabel("Time (ps)")
    axes[0].get_legend().remove()
    axes[0].set_ylabel("")
    axes[0].set_title("residual 1st LSV")
    custom_cycler = cycler(color=[colors[0]])
    plot_rsv_residual(result_dataset, axes[1], indices=[0], cycler=custom_cycler)
    custom_cycler = cycler(color=[colors[1]])
    plot_rsv_residual(result_dataset2, axes[1], indices=[0], cycler=custom_cycler)
    custom_cycler = cycler(color=[colors[2]])
    plot_rsv_residual(result_dataset3, axes[1], indices=[0], cycler=custom_cycler)
    axes[1].set_xlabel("Wavelength (nm)")
    axes[1].set_title("residual 1st RSV")
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")

    return fig, axes


fig, axes = plot_svd_of_residual(
    target_result1.data["CF9212WLtr1"],
    target_result1.data["CF9212WLtr2"],
    target_result1.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

The residuals are highly structured, especially around the location of the maximum of the IRF. Therefore we change the IRF shape to a double Gaussian.

In [ ]:
target_result1.root_mean_square_error

## Step 1c Create scheme with double Gaussian IRF and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/globalWLstep1c_PSI_CF9212_streak_global_dispWLdouble.yml",
    parameters="models/globalWLstep1c_PSI_CF9212_streak_global_dispWLdouble.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result2 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 7         1.7473e+02


In [ ]:
target_result2

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result2.data["CF9212WLtr1"],
    target_result2.data["CF9212WLtr2"],
    target_result2.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

The residuals are still structured, therefore we know introduce an independent free Chla fraction for each time range.

In [ ]:
target_result2.root_mean_square_error

## Step 1d Create scheme with free Chl a fraction per experiment

In [ ]:
target_scheme = Scheme(
    model="models/globalWLstep1d_PSI_CF9212_streak_global_dispWLfree.yml",
    parameters="models/globalWLstep1d_PSI_CF9212_streak_global_dispWLfree.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result3 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 7         1.4290e+02 


In [ ]:
target_result3

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result3.data["CF9212WLtr1"],
    target_result3.data["CF9212WLtr2"],
    target_result3.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

In [ ]:
target_result3.root_mean_square_error

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode

# ColorCode.green,"k",
myFRLcolors = [
    "k",
    "r",
    "b",
    ColorCode.green,
    "tab:orange",
    "tab:grey",
    ColorCode.cyan,
    "w",
]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [
    "g",
    "tab:orange",
    "r",
    "k",
    ColorCode.magenta,
    ColorCode.indigo,
    ColorCode.brown,
    "w",
    "w",
    "w",
    "y",
    ColorCode.maroon,
]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result3.data["CF9212WLtr1"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result3.data["CF9212WLtr2"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result3.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result3.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result3.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result3.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
plot_das(target_result3.data["CF9212WLtr4"], axes[2], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("EAS/SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
show_a_matrixes(target_result3)

In [ ]:
target_result3.optimized_parameters

### Compute the FWHM of the main Gaussian of the IRF

In [ ]:
import numpy as np

const = 2 * np.sqrt(2 * np.log(2))
[
    const * target_result3.optimized_parameters.get("CF9212irfWLtr1.width1").value,
    const * target_result3.optimized_parameters.get("CF9212irfWLtr2.width1").value,
    const * target_result3.optimized_parameters.get("CF9212irfWLtr4.width1").value,
]

### Fit quality of the global analysis of the WL data
overlays of traces and fits of 6 selected wavelengths

In [ ]:
CF9212target_result_streak = (
    # target_result.data["CF9212FRLtr1"],
    target_result3.data["CF9212WLtr1"],
    # target_result.data["CF9212FRLtr2"],
    target_result3.data["CF9212WLtr2"],
    # target_result.data["CF9212FRLtr4"],
    target_result3.data["CF9212WLtr4"],
)

In [ ]:
import numpy as np

# wavelengths = select_plot_wavelengths(
#     CF9212target_result_streak, equidistant_wavelengths=True
# )
wavelengths = np.linspace(665, 795, num=16)
plot_fitted_traces(CF9212target_result_streak, wavelengths, linlog=True, linthresh=100)

In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        CF9212target_result_streak,
        [685, 705, 715, 740],
        linlog=True,
        linthresh=10,
        axes_shape=(3, 2),
        figsize=(8, 6),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "TR1"
        elif i == 3:
            labels[i] = "TR2"
        elif i == 5:
            labels[i] = "TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        CF9212target_result_streak,
        [685, 705, 715, 740],
        linlog=True,
        linthresh=10,
        axes_shape=(2, 2),
        figsize=(8, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "TR1"
        elif i == 3:
            labels[i] = "TR2"
        elif i == 5:
            labels[i] = "TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

## Step 2a Create simple target scheme of WL-PSI with Bulk and RP

In [ ]:
# Uncomment the following 3 lines to display the target model file in the notebook
from glotaran.utils.ipython import display_file

target_model_path = "models/targetWLstep2a_PSI_CF9212_streak_target_dispWL.yml"
display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

In [ ]:
target_scheme = Scheme(
    model="models/targetWLstep2a_PSI_CF9212_streak_target_dispWL.yml",
    parameters="models/targetWLstep2a_PSI_CF9212_streak_target_dispWL.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
        # "WLRCSAS": "data/PSI_7521FR_WLtr124av8nobridgetarget_dispcorri4WLRC.ascii",
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result = optimize(target_scheme, raise_exception=True)

In [ ]:
target_result

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result.data["CF9212WLtr1"],
    target_result.data["CF9212WLtr2"],
    target_result.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

Note the very large residuals

In [ ]:
target_result.root_mean_square_error

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# ColorCode.green,"k",
myFRLcolors = ["g", ColorCode.cyan, ColorCode.green, "w"]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
custom_cycler = cycler(color=myFRLcolors)
# plot_concentrations(target_result.data["CF9212WLtr1"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(
    target_result.data["CF9212WLtr2"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Step 2b Create target scheme of WL-PSI with Bulk, Red and RP


In [ ]:
target_scheme = Scheme(
    model="models/targetWLstep2b_PSI_CF9212_streak_target_dispWL.yml",
    parameters="models/targetWLstep2b_PSI_CF9212_streak_target_dispWL.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
        # "WLRCSAS": "data/PSI_7521FR_WLtr124av8nobridgetarget_dispcorri4WLRC.ascii",
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result = optimize(target_scheme, raise_exception=True)

In [ ]:
target_result

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result.data["CF9212WLtr1"],
    target_result.data["CF9212WLtr2"],
    target_result.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

Note the still large trends in the residuals, especially in the 1st RSV around 730 nm.

In [ ]:
target_result.root_mean_square_error

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# ColorCode.green,"k",
myFRLcolors = ["g", "r", ColorCode.cyan, ColorCode.green, "w"]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
custom_cycler = cycler(color=myFRLcolors)
# plot_concentrations(target_result.data["CF9212WLtr1"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(
    target_result.data["CF9212WLtr2"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Step 2c Create target scheme of WL-PSI with Bulk, Red1, Red2 and RP


In [ ]:
target_scheme = Scheme(
    model="models/targetWLstep2c_PSI_CF9212_streak_target_dispWL.yml",
    parameters="models/targetWLstep2c_PSI_CF9212_streak_target_dispWL.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
        # "WLRCSAS": "data/PSI_7521FR_WLtr124av8nobridgetarget_dispcorri4WLRC.ascii",
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result = optimize(target_scheme, raise_exception=True)

In [ ]:
target_result

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result.data["CF9212WLtr1"],
    target_result.data["CF9212WLtr2"],
    target_result.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

Note that the previous trends in the residuals, especially in the 1st RSV around 730 nm, have disappeared.

In [ ]:
target_result.root_mean_square_error

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# ColorCode.green,"k",
myFRLcolors = ["g", "tab:orange", "r", ColorCode.cyan, ColorCode.green, "w"]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
custom_cycler = cycler(color=myFRLcolors)
# plot_concentrations(target_result.data["CF9212WLtr1"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(
    target_result.data["CF9212WLtr2"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Step 2d Create target scheme of WL-PSI with a priori knowledge
In the target analysis of WL-PSI we use a kinetic scheme (see Figure 10A in the paper) that is based upon the a priori knowledge from the transient absorption experiments about the mechanism of charge separation in WL-PSI. The SAS of the RC is shifted to longer wavelengths relative to the Bulk Chl a SAS. To estimate the RC SAS we will use a guidance SAS. 

In [ ]:
# Uncomment the following 3 lines to display the target model file in the notebook
from glotaran.utils.ipython import display_file

target_model_path = "models/targetWLstep2d_PSI_CF9212_streak_target_dispWL.yml"
display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

In [ ]:
target_scheme = Scheme(
    model="models/targetWLstep2d_PSI_CF9212_streak_target_dispWL.yml",
    parameters="models/targetWLstep2d_PSI_CF9212_streak_target_dispWL.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212WLtr4": CF9212DATA_PATH6,
        "WLRCSAS": "data/PSI_7521FR_WLtr124av8nobridgetarget_dispcorri4WLRC.ascii",
    },
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 7         1.4475e+02


In [ ]:
target_result

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result.data["CF9212WLtr1"],
    target_result.data["CF9212WLtr2"],
    target_result.data["CF9212WLtr4"],
    linlog=True,
    linthresh=100,
    loc1=57,
    loc2=57,
    loc3=214,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

In [ ]:
target_result.root_mean_square_error

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# ColorCode.green,"k",
myFRLcolors = ["g", "tab:orange", "r", "tab:grey", ColorCode.cyan, ColorCode.green, "w"]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [
    "g",
    "tab:orange",
    "r",
    "k",
    ColorCode.magenta,
    ColorCode.indigo,
    ColorCode.brown,
    "w",
    "w",
    "w",
    "y",
    ColorCode.maroon,
]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
custom_cycler = cycler(color=myFRLcolors)
# plot_concentrations(target_result.data["CF9212WLtr1"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(
    target_result.data["CF9212WLtr2"],
    axes[0],
    center_λ=0,
    linlog=True,
    linthresh=100,
    cycler=custom_cycler,
)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
show_a_matrixes(target_result)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 40MB of disk space per save.

In [ ]:
save_result(
    result=target_result,
    result_path="results/20230912targetWL_disp/result.yaml",
    allow_overwrite=True,
)

### Create the WL guidance data sets

In [ ]:
from glotaran.io import save_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in target_result.data["CF9212WLtr2"].species:
    clp_guide = create_clp_guide_dataset(
        target_result.data["CF9212WLtr2"], species.item()
    )
    string_in_string = "guide/20230912PSI_CF9212_streak_targetWL_disp_{}.ascii".format(
        species.item()
    )
    save_dataset(clp_guide.data, string_in_string, allow_overwrite=True)

### Print the estimates of the optimized parameters and their precision 
The t-values of the **free** parameters (Vary=True) indicate the precision

In [ ]:
target_result.optimized_parameters

### K-matrix target

In [ ]:
compartments = target_scheme.model.initial_concentration["CF9212inputWL"].compartments

target_scheme.model.k_matrix["CF9212skmBulkWL"].matrix_as_markdown(
    compartments
).replace("0.0000e+00", "")

#### Overview plots per dataset

### WL data

NB plot_overview with linlog=True is very slow, but it can be more informative because it zooms in on the early events

In [ ]:
from cycler import cycler

fig, axes = plot_overview(
    target_result.data["CF9212WLtr1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    # linlog=True,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

In [ ]:
from cycler import cycler

fig, axes = plot_overview(
    target_result.data["CF9212WLtr2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    # linlog=True,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

In [ ]:
from cycler import cycler

plot_overview(
    target_result.data["CF9212WLtr4"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

### Fit quality of the target analysis of the WL data
overlays of traces and fits, first of 16 wavelengths, then of 6 selected wavelengths

In [ ]:
CF9212target_result_streak = (
    # target_result.data["CF9212FRLtr1"],
    target_result.data["CF9212WLtr1"],
    # target_result.data["CF9212FRLtr2"],
    target_result.data["CF9212WLtr2"],
    # target_result.data["CF9212FRLtr4"],
    target_result.data["CF9212WLtr4"],
)

In [ ]:
import numpy as np

# wavelengths = select_plot_wavelengths(
#     CF9212target_result_streak, equidistant_wavelengths=True
# )
wavelengths = np.linspace(665, 795, num=16)
plot_fitted_traces(CF9212target_result_streak, wavelengths, linlog=True, linthresh=100)

In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        CF9212target_result_streak,
        [685, 705, 715, 740, 750, 795],
        linlog=True,
        linthresh=10,
        axes_shape=(3, 2),
        figsize=(8, 6),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "TR1"
        elif i == 3:
            labels[i] = "TR2"
        elif i == 5:
            labels[i] = "TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()